## 대용량 비정형 데이터 기반 Advanced RAG 구축 및 평가
#### 강의 목표
1. **Real-World Data Handling**: 나무위키(구어체/지식) FinePDFs(문어체/전문문서)가 섞인 두개 데이터를 처리합니다.
2. **Beyond Keyword Search**: 단순 키워드 매칭(BM25)과 의미 기반 검색(Vector Search)의 결정적 차이를 체감합니다.
3. **Hybrid Search**: BM25와 Vector Search를 결합하여 검색 성능을 향상시킵니다.
4. **Advanced Pipeline**: Retriever + Reranker 구조를 통해 검색 정확도를 극대화하는 기법을 익힙니다.
5. **Query Decomposition**: 복잡한 질의를 분해하여 더 정확한 검색 결과를 얻는 방법을 배웁니다.
6. **Data-Driven Evalution**: 정답지가 없는 상황에서 LLM을 이용해 *합성 데이터셋(Synthetic Testset)*을 만들고 평가를 수행합니다.

#### 사용 모델 및 데이터
- **LLM**: gpt-5-mini (High Performance & Low Cost)
- **Data**: heegyu/namuwiki (General Knowledge) + HuggingFaceFW/finepdfs-edu (Domain Documents)

In [1]:
# 1. 필수 라이브러리 설치 (Jupyter 환경 권장 방식)

# 버전을 명시하여 호환성 문제 방지

%pip install -q \
    langchain==0.3.14 \
    langchain-openai==0.2.14 \
    langchain-huggingface==0.1.2 \
    langchain-community==0.3.14 \
    ragas==0.2.10 \
    datasets \
    sentence-transformers \
    tiktoken \
    rank_bm25 \
    pandas \
    faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import warnings
from dotenv import load_dotenv

warnings.filterwarnings('ignore')

# API_KEY 정보 로드
load_dotenv()


True

In [ ]:
# print API key
# print(f"[API KEY]\n{os.environ['OPENAI_API_KEY'][:-15]}" + "*" * 15)

## 1. Data Loading: '현실적인' 데이터 믹싱(Mixing)
현실에는 깔끔하게 정제된 데이터만 존재하지 않습니다. **지식 백과(Wiki)** 스타일과 **보고서(PDF)** 스타일의 텍스트가 섞여서, 난이도가 높은 통합 검색 환경을 구축합니다.

In [4]:
from datasets import load_dataset
from langchain.docstore.document import Document

print("📥 데이터 스트리밍 시작...")

raw_docs = []

# [Source A] 나무위키 (General Knowledge)
# 특징: 구어체, 다양한 주제, 메타데이터에 제목 포함
wiki_stream = load_dataset("heegyu/namuwiki", split="train", streaming=True)
for idx, data in enumerate(wiki_stream):
    if idx >= 100: break  # 실습 속도를 위해 100개만 사용, Memory & Computation 여유가 있다면 전부 써보기
    # print(f"[{idx+1}/100] {data}")
    content = f"[출처: 나무위키] 제목: {data['title']}\n\n{data['text']}"
    raw_docs.append(Document(page_content=content, metadata={"source": "namuwiki", "title": data['title']}))

length_of_wiki = len(raw_docs)

# for i, doc in enumerate(raw_docs):
#     print(f"[{i+1}/10] {doc}")

# [Source B] FinePDFs-Edu (Domain Documents)
# 특징: 문어체, 전문용어, 서식(Header/Footer) 노이즈 존재
pdf_stream = load_dataset("HuggingFaceFW/finepdfs-edu", "kor_Hang", split="train", streaming=True)
for idx, data in enumerate(pdf_stream):
    if idx >= 100: break
    # PDF는 제목이 명확하지 않은 경우가 많아 URL을 메타데이터로 사용
    raw_docs.append(Document(page_content=data['text'], metadata={"source": "finepdf", "url": data['url']}))

length_of_pdf = len(raw_docs) - length_of_wiki

print(f"✅ 데이터 로드 완료: 총 {len(raw_docs)}개 문서")
print(f"   - Wiki 예시(총 {length_of_wiki}개): {raw_docs[0].page_content[:80]}...")
print(f"   - PDF 예시(총 {length_of_pdf}개): {raw_docs[length_of_wiki].page_content[:80]}...")




📥 데이터 스트리밍 시작...
✅ 데이터 로드 완료: 총 200개 문서
   - Wiki 예시(총 100개): [출처: 나무위키] 제목: !

#redirect 느낌표
...
   - PDF 예시(총 100개): 이해도 테스트

타입ㆍ제조법

Q1 사케는 무엇으로 제조되는가?

a) 사과
b) 쌀
c) 보리
d) 콩

Q2 일본에서 사케를 제조하고 있는 ...


---

## 2. Advanced Chunking Strategy
PPT에서 강조한 **Chunking** 전략입니다.
- **Attention Dilution 방지**: 너무 길면 LLM이 핵심을 놓칩니다.
- **Context Overlap**: 문맥이 끊기는 것을 방지하기 위해 Chunk_overlap을 둡니다.
- **Separators**: 문단(\n\n) → 줄바꿈(\n) → 문장(.) 순서로 의미 단위를 보존하며 자릅니다.

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# PPT 실무 팁 반영: 의미 단위 보존을 위한 계층적 Split
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,  # 검색에 적절한 크기 (토큰 기준 아님, 글자 수 기준)
    chunk_overlap=100,  # 앞뒤 문맥 연결을 위한 오버랩
    separators=["\n\n", "\n", ". ", " ", ""]  # 의미 단위 우선 순위
)

splitted_docs = text_splitter.split_documents(raw_docs)
print(f"✂️ 청킹 결과: {len(raw_docs)}개 문서 → {len(splitted_docs)}개 청크로 분할됨")

✂️ 청킹 결과: 200개 문서 → 7641개 청크로 분할됨


---

## 3. 검색 엔진 구축: BM25, Vector, Hybrid
세가지 검색 방식을 구축하고 비교합니다.

|방식|특징|장점|단점|
|----|----|----|----|
|**BM25**|키워드 매칭|정확한 용어 검색에 강함|동의어/유사어 인식 불가|
|**Vector**|의미 기반|문맥적 유사성 파악|특정 키워드 검색에 약함|
|**Hybrid**|BM25 + Vector 결합|두 장점 결합|가중치 튜닝 필요|


In [6]:
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 1. BM25 Retriever 구축(키워드 기반)
bm25_retriever = BM25Retriever.from_documents(splitted_docs)
bm25_retriever.k = 5  # 후보군을 넉넉하게

# 2. Vector Store 구축 (의미 기반, FAISS 사용)
# OpenAI Embedding 사용
openai_embedding = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = FAISS.from_documents(
    documents=splitted_docs,
    embedding=openai_embedding
)
vector_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

print("✅ BM25 & Vector(FAISS) 검색 엔진 준비 완료!")

✅ BM25 & Vector(FAISS) 검색 엔진 준비 완료!


#### 3.1 🆕 Hybrid Search 구현 (PPT 23-24 슬라이드)
**λ•BM25 + (1-λ)•Vector** 하이브리드 검색입니다.
```
최종 스코어 = α * BM25_score + (1 - α) * Vector_score
```
LangChain의 EnsembleRetriever를 사용하여 두 검색 결과를 RRF(Reciprocal Rank Fusion) 방식으로 결합합니다.

In [7]:
from langchain.retrievers import EnsembleRetriever

# Hybrid Retriever: BM25(키워드) + Vector(의미) 결합
# weights: [BM25 가중치, Vector 가중치] - 합이 1이 될 필요는 없음
hybrid_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=[0.4, 0.6]  # Vector에 약간 더 가중치 (사전 정의된 질문 Coverage에 따라 조정)
)

print("✅ Hybrid Search 엔진 준비 완료!")
print("   - BM25 가중치: 0.4 (키워드 매칭)")
print("   - Vector 가중치: 0.6 (의미 기반)")

✅ Hybrid Search 엔진 준비 완료!
   - BM25 가중치: 0.4 (키워드 매칭)
   - Vector 가중치: 0.6 (의미 기반)


#### 3.2 검색 방식 비교 실험
동일한 질문에 대해 세가지 검색 방식의 결과를 비교해봅니다.

In [9]:
import pandas as pd

def compare_retrievers(query: str):
    """세가지 검색 방식 결과 비교"""
    print(f"🔍 질문: '{query}'\n")
    print("=" * 80)

    results = {
        "BM25 (키워드)": bm25_retriever.invoke(query),
        "Vector (의미)": vector_retriever.invoke(query),
        "Hybrid (결합)": hybrid_retriever.invoke(query)
    }

    for method, docs in results.items():
        print(f"\n📌 [{method}] 상위 3개 결과:")
        print("-" * 40)
        for i, doc in enumerate(docs[:3], 1):
            source = doc.metadata.get('source', 'unknown')
            preview = doc.page_content[:100].replace('\n', ' ')
            print(f"   {i}. [{source}] {preview}...")
        if not docs:
            print("   → 결과 없음")
    
    return results

# 테스트 1: 의미론적 질문(Vector가 유리)
results1 = compare_retrievers("프린터 추천")


🔍 질문: '프린터 추천'


📌 [BM25 (키워드)] 상위 3개 결과:
----------------------------------------
   1. [finepdf] 인쇄 - 로컬 프린터 / 네트워크 프린터 설치, 여러대 프린터 설치 가능; 기본프린터는 대량설치 (로컬 네트워크로 설치한 프린터 모두 기본프린터로 설정 가능)   - 문서인쇄: 해...
   2. [finepdf] 프린터 공유 - 시장 - 잠재업 프린터 - 공유한 프린터 - 속성 - 공유 - '어플리케션 공유'   - 시장 - 제어판 - 네트워크 및 공유 센터 - 공유 설정 변경 - 서비스 ...
   3. [finepdf] 수선화시낭송 동아리  코로나19의 장기화로 함께하는 동아리 모임, 찾아가는 봉사 활동이 어려워진 환경 속에서 다양하고 혁신적인 온라인 교육 활동을 개발하여 시낭송의 의미와 즐거움이...

📌 [Vector (의미)] 상위 3개 결과:
----------------------------------------
   1. [finepdf] 프린터 공유 - 시장 - 잠재업 프린터 - 공유한 프린터 - 속성 - 공유 - '어플리케션 공유'   - 시장 - 제어판 - 네트워크 및 공유 센터 - 공유 설정 변경 - 서비스 ...
   2. [finepdf] 자주쓰이기  컴퓨터 → 드라이브 → 속성, 디스크정리, → 공간확보개별.. PC 관리 구분 필요 레지스트리 → 부팅만 아니라 운영이 필요한 정보가있 프린팅기는 윈도우에서 이용.  ...
   3. [finepdf] 보조프로그램 - 메모장, 워드프레스 프로그램 등 - 앱 프로그램 (네이버, 마이크로소프트) - 기타 워드프레스 (FTP, 바이너스 프로그램, 뷰어 프로그램)  질문  요약 스플 -...

📌 [Hybrid (결합)] 상위 3개 결과:
----------------------------------------
   1. [finepdf] 프린터 공유 - 시장 - 잠재업 프린터 - 공유한 프린터 - 속성 - 공유 - '

In [10]:
# 테스트 2: 특정 키워드 질문 (BM25가 유리)
results2 = compare_retrievers("대한민국 헌법 제1조")

🔍 질문: '대한민국 헌법 제1조'


📌 [BM25 (키워드)] 상위 3개 결과:
----------------------------------------
   1. [finepdf] ‘IMF’와 ‘벤처기업’과 눈먼 나라 돈 이대로 ................................................................. 11 4월 1...
   2. [finepdf] 중소기업을 육성시켜서 일자리를 늘리고 우리 경제를 발전시키겠다는 취지는 공감하지만 ‘벤처’란 외국말을 대한민국 중앙부처 이름에 들어가는 것은 강력하게 반대합니다.  그 까닭은 우리...
   3. [namuwiki] 대한민국 국군에서도 M1911을 제식으로 채용 했었으나 [[K5 권총]]으로 전부 교체했다. == 관련 문서 ==  * [[무기 관련 정보]]  * [[각종 탄약 포탄 구경 일람]...

📌 [Vector (의미)] 상위 3개 결과:
----------------------------------------
   1. [finepdf] ⑥ 직접원인은 인적원인과 물적원인으로 구별된다.  ⑦ 직접원인(1차원인)에는 그것의 존재 이유가 있다. 이것을 2차원인 이라고 한다.  ⑧ 2차원인 이전에는 기초원인이 있다.  ⑨...
   2. [finepdf] 4 가지 원칙...
   3. [finepdf] 첫째로 적용된 병법은 '정보획득(情報獲得)의 원리'이다. 전라좌우수영과 경상우수영 함대를 통합한 이순신 함대는 한산해전이 벌어지기 하루 전인 임진년(1592 년) 7 월 6 일 미...

📌 [Hybrid (결합)] 상위 3개 결과:
----------------------------------------
   1. [finepdf] ⑥ 직접원인은 인적원인과 물적원인으로 구별된다.  ⑦ 직접원인(1차원인)에는 그것의 존재 이유가 있다. 이것을 2차원인 이라고 한다.  ⑧ 2차원인 이전에는 기초원인이 있다.  ⑨...
   2. [finepdf] 4 가지 원칙...
 

In [11]:
# 테스트 3: 복합적인 질문 (Hybrid가 유리)
results3 = compare_retrievers("인공지능 기술의 발전과 미래 전망")

🔍 질문: '인공지능 기술의 발전과 미래 전망'


📌 [BM25 (키워드)] 상위 3개 결과:
----------------------------------------
   1. [finepdf] 설문조사는 어떤 주제를 다루나요?  설문조사는 학생들이 학교에서 겪는 웰빙, 참여 및 경험과 관련된 다양한 측면들을 조사합니다. 질문에 포함되는 내용은 다음과 같습니다:  회복력 ...
   2. [finepdf] 관련한 정책들을 다룬다.  l 한국정치특강(Korean Politics Seminar Study) 한국정치의 과거와 현재를 조명하고 인지하며, 미래 한국정치의 발전방향을 논의하고 ...
   3. [finepdf] 이런 책은 읽지 맙시다  이정우 이오덕학교 큰 선생님.  오래된 미래 라다크로부터(에서) 배운다. 헬레나 노르베리-호지 녹색평론 출판사, 김종철/ 김태언 옮김...

📌 [Vector (의미)] 상위 3개 결과:
----------------------------------------
   1. [finepdf] . 왜냐하면 '의미'나 '목적'은 아무리 인공지능이 발달한다 해도 자동화 될 수 없고, '진 심' 또한 마찬가지 이기 때문이다. 저자는 우리에게 이러한 능력이 있기에, 공상과학에서...
   2. [finepdf] 물론 이러한 어두운 전망에는 그럴만한 이유가 있다. 심각한 환경 변화와 사회적 문제들이 나타나고 있다. 특히 기후변화와 불평등, 인구변화가 특징적이다. 이러한 현상들에 인공지능과 ...
   3. [finepdf] (6) 아이들의 미래를 위한 디지털 활용 능력 향상 지원  학생들의 디지털 역량 강화 교육도 확대한다. 초중고 학생들에게 민·관의 우수 소프트웨어, 인공지능 등의 교육을 제공하는 ...

📌 [Hybrid (결합)] 상위 3개 결과:
----------------------------------------
   1. [finepdf] . 왜냐하면 '의미'나 '목적'은 아무리 인공지능이 발달한다 해도 자동화 될 수 

---

## 4. Advanced Pipeline: Reranker 도입
"Rerank의 개념"
|단계|모델|특징|
|----|----|----|
|**1차 검색(Retriever)**|Bi-Encoder|빠름, 대량 후보 추출|
|**2차 정렬(Reranker)**|Cross-Encoder|느림, 정밀한 관련성 평가|

이 **2-Stage Retrieval** 구조가 현업에서 가장 많이 쓰이는 고성능 패턴입니다.

In [12]:
# # --- 온프렘 CrossEncoder Reranker (직접 모델 로드, 참고용) ---
# from sentence_transformers import CrossEncoder
# from langchain.retrievers.document_compressors import BaseDocumentCompressor
# from langchain.schema import Document
# from typing import List, Sequence
# from pydantic import PrivateAttr

# class CrossEncoderReranker(BaseDocumentCompressor):
#     """Cross-Encoder 기반 Reranker (온프렘)"""
#     model_name: str = "BAAI/bge-reranker-base"
#     top_n: int = 3
#     _model: CrossEncoder = PrivateAttr()

#     def __init__(self, **kwargs):
#         super().__init__(**kwargs)
#         self._model = CrossEncoder(self.model_name)
    
#     def compress_documents(self, documents: Sequence[Document], query: str, callbacks=None) -> List[Document]:
#         if not documents:
#             return []
#         pairs = [(query, doc.page_content) for doc in documents]
#         scores = self._model.predict(pairs)
#         scored_docs = list(zip(scores, documents))
#         scored_docs.sort(key=lambda x: x[0], reverse=True)
#         return [doc for _, doc in scored_docs[:self.top_n]]
    
# # 온프렘 Reranker 사용 예시 (참고용)
# reranker = CrossEncoderReranker(model_name="BAAI/bge-reranker-base", top_n=3)

# # --- Cohere Rerank API 직접 호출 커스텀 Reranker ---
import cohere
from langchain_core.documents.compressor import BaseDocumentCompressor
from langchain.schema import Document
from typing import List, Sequence
from pydantic import PrivateAttr

class CohereReranker(BaseDocumentCompressor):
    cohere_api_key: str
    top_n: int = 3
    model: str = "rerank-multilingual-v3.0"
    _client: cohere.Client = PrivateAttr()

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._client = cohere.Client(self.cohere_api_key)
    
    def compress_documents(self, documents: Sequence[Document], query: str, callbacks=None) -> List[Document]:
        if not documents:
            return []
        docs_text = [doc.page_content for doc in documents]
        results = self._client.rerank(
            model=self.model,
            query=query,
            documents=docs_text,
            top_n=self.top_n
        )
        return [documents[r.index] for r in results.results]

# 사용 예시
cohere_api_key = os.environ['COHERE_API_KEY']
reranker = CohereReranker(cohere_api_key=cohere_api_key, top_n=3)

# Base Retriever: Hybrid에서 후보군을 넉넉하게(K=10) 가져옴
from  langchain.retrievers import EnsembleRetriever
hybrid_retriever_wide = EnsembleRetriever(
    retrievers=[
        BM25Retriever.from_documents(splitted_docs, k=10),
        vectorstore.as_retriever(search_kwargs={"k": 10})
    ],
    weights=[0.4, 0.6]
)

# 최종 Rerank Pipeline
from langchain.retrievers import ContextualCompressionRetriever
rerank_retriever = ContextualCompressionRetriever(
    base_compressor=reranker,
    base_retriever=hybrid_retriever_wide
)

print("✅ Advanced RAG Pipeline (Cohere Reranker, 직접 호출) 구축 완료!")
print("   - 1차: Hybrid Search로 20개 후보 추출")
print("   - 2차: Cohere Rerank로 상위 3개 재정렬")


ModuleNotFoundError: No module named 'cohere'

#### 5. 🆕 Query Decomposition (PPT 41-43 슬라이드)
**쿼리 분해(Query Decomposition)** 기법

복잡한 질문을 여러 개의 단순한 하위 질문으로 분해하여:
1. 각 하위 질문에 대해 검색 수행
2. 결과를 종합하여 최종 답변 생성
**예시**:
- 원본: "삼성전자랑 하이닉스 가격 상승 요인 비교해봐"
- 분해:
    - "삼성전자 가격 상승 요인"
    - "하이닉스 가격 상승 요인"

In [ ]:
from openai import OpenAI

client = OpenAI()  # OPENAI_API_KEY 환경변수 사용

def decompose_query(query: str) -> list:
    """복잡한 쿼리를 하위 쿼리들로 분해"""
    response = client.chat.completions.create(
        model="gpt-4.1",
        temperature=0,
        messages=[
            {
                "role": "user",
                "content": f"""당신은 복잡한 질문을 하위 질문들로 분해하는 전문가입니다.
주어진 질문을 검색에 적합한 2~4개의 하위 질문으로 분해해주세요.
각 하위 질문은 독립적으로 검색 가능해야 합니다.

원본 질문: {query}

하위 질문들 '한 줄에 하나씩, 번호 없이):"""
            }
        ]
    )

    result = response.choices[0].message.content
    sub_queries = [q.strip() for q in result.strip().split('\n') if q.strip()]
    return sub_queries

# 테스트
complex_query = "삼성전자와 SK하이닉스의 반도체 기술력과 시장 점유율을 비교 분석해줘."
sub_queries = decompose_query(complex_query)

print(f"📝 원본 질문:{complex_query}")
print(f"\n🔀 분해된 하위 질문들:")
for i, sq in enumerate(sub_queries, 1):
    print(f"    {i}. {sq}")

In [ ]:
def retrieve_with_decomposition(query: str, retriever, top_k: int = 3) -> list:
    """
    쿼리 분해 + 검색 + 결과 통합 파이프라인
    """
    # 1. 쿼리 분해
    sub_queries = decompose_query(query)
    print(f"🔀 '{query[:30]}...' → {len(sub_queries)}개 하위 쿼리로 분해")

    #2. 각 화위 쿼리에 대해 검색
    all_docs = []
    seen_contents = set()

    for sq in sub_queries:
        docs = retriever.invoke(sq)
        for doc in docs:
            # 중복 제거
            content_hash = hash(doc.page_content[:200])
            if content_hash not in seen_contents:
                seen_contents.add(content_hash)
                all_docs.append(doc)
    
    print(f"🗒️ 총 {len(all_docs)}개 고유 문서 검색됨")

    # 3. Reranker로 최종 정렬 (선택적)
    if len(all_docs) > top_k:
        final_docs = reranker.compress_documents(all_docs, query)
    else:
        final_docs = all_docs[:top_k]
    
    return final_docs

# 테스트
print("\n" + "=" * 80)
docomposed_results = retrieve_with_decomposition(
    "인공지능의 장단점과 미래 발전 방향에 대해 설명해줘",
    hybrid_retriever
)

print("\n📌 최종 검색 결과:")
for i, doc in enumerate(decomposed_results, 1):
    print(f"   {i}. [{doc.metadata.get('source', 'unknown')}] {doc.page_content[:80]}...")


---

#### 6. RAG Chain 구성 및 테스트

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

# 최종 Generation Chain 연결

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=rerank_retriever,
    return_source_documents=True
)

# 테스트
response = rag_chain.invoke({"query": "한국의 교육 제도에 대해 설명해줘"})
print(f"🤖 답변:\n{response['result']}")
print(f"\n🗒️ 참조한 문서 출처: {[doc.metadata.get('source', 'unknown') for doc in response['source_documents']]}")

---

#### 7. Synthetic Testset Generation (평가 데이터 생성)

평가 파트 입니다.(without RAGAS)

"정답지(Ground Truth)가 없는데 어떻게 평가하나요?" → **LLM을 시켜서 문서 기반으로 문제를 출제하면 됩니다.**

문서 내용을 바탕으로 '질문'과 '모범 답안'을 자동 생성합니다.

In [ ]:
from openai import OpenAI
import json
import pandas as pd

client = OpenAI()

def generate_test_questions(docs, num_questions=5):
    """ 문서에서 테스트 질문/답변 쌍 생성"""

    # 문서 내용 합치기
    combined_text = "\n\n".join([doc.page_content for doc in docs[:30]])

    # 텍스트가 너무 길면 자르기
    if len(combined_text) > 15000:
        combined_text = combined_text[:15000]
    
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        temperature=0.7,
        messages=[
            {
                "role": "system",
                "content": "당신은 문서를 읽고 테스트 문제를 출제하는 전문가입니다."
            },
            {
                "role": "user",
                "content": f"""다음 문서를 읽고 {num_questions}개의 질문과 답변을 만들어주세요.

문서:
{combined_text}

다음 JSON 형식으로만 응답해주세요. (다른 텍스트 없이)
[
    {{"question": "질문1", "answer": "답변1"}},
    {{"question": "질문2", "answer": "답변2"}}
]
"""
            }
        ]
    )

    result = response.choices[0].message.content

    # JSON 파싱
    try:
        # 혹시 ```json``` 같은 마크다운이 있으면 제거
        result = result.strip()
        if result.startswith("```"):
            result = result.split("```")[1]
            if result.startswith("json"):
                result = result[4:]
        
        qa_pairs = json.loads(result)
        return qa_pairs
    except:
        print("⚠️ JSON 파싱 실패. 원본 응답:")
        print(result)
        return []

# 실행
print("🤖 AI가 문서를 읽고 시험 문제를 출제 중입니다...")
print("    ⌛ 잠시만 기다려주세요...")

qa_pairs = generate_test_questions(splitted_docs, num_questions=5)

# DataFrame으로 변환
test_df = pd.DataFrame(qa_pairs)
test_df.columns = ['user_input', 'reference']

print(f"\n✅ {len(test_df)}개의 테스트 문제가 생성되었습니다!")
display(test_df)

---

#### 8. 🆕 Evaluation (정량 평가) + 메트릭 해석 가이드

PPT 56번 슬라이드의 주요 평가 메트릭을 실제로 측정합니다. 다만 심플함을 위해 llm-as-judge를 사용해봅시다!

#### 📊 RAGAS 핵심 메트릭 해석 가이드

|메트릭|측정대상|의미|낮을 때 원인|
|----|----|----|----|
|**Faithfulness**|생성 모듈|답변이 검색된 Context에 **충실**한가? (할루시네이션 체크)|LLM이 Context 외 정보로 답변 생성|
|**Answer Relevancy**|생성 모듈|답변이 질문의 **의도에 부합**하는가?|질문과 무관한 내용 포함|
|**Context Recall**|검색 모듈|정답에 필요한 정보가 Context에 **포함**되었는가?|Retriever가 관련 문서 누락|
|**Context Precision**|검색 모듈|검색된 Context가 **관련성 높은 순서**로 정렬되었는가?|노이즈 문서가 상위에 위치|

#### 🎯 목표 점수 가이드라인
- **0.8 이상**: 우수 (Production Ready)
- **0.6-0.8**: 양호 (개선 여지 있음)
- **0.6 미만**: 개선 필요

In [ ]:
# RAGAS 없이 간단한 평가 (LLM 기반)
from openai import OpenAI
import pandas as pd

client = OpenAI()

def simple_evalute(question, answer, reference, context):
    """LLM을 이용한 간단한 평가"""
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        temperature=0,
        messages=[{
            "role": "user",
            "content": f"""다음 RAG 시스템의 답변을 평가해주세요.

질문: {question}
정답(참조): {reference}
RAG 답변: {answer}
검색된 문서: {context[:500]}...

다음 기준으로 1-5점 평가 (JSON으로만 응답):
- faithfulness: 답변이 검색 문서에 충실한가?
- relevancy: 답변이 질문에 적절한가?
-  correctness: 답변이 정답과 일치하는가?

{{"faithfulness": 점수, "relevancy": 점수, "correctness": 점수, "comment": "한줄평"}}"""
        }]
    )
    return response.choices[0].message.content

# 평가 실행
print("📊 RAG 평가 진행 중...")
results = []

for i, row in test_df.iterrows():
    q = row['user_input']
    ref = row['reference']

    # RAG로 답변 생성
    rag_result = rag_chain.invoke(["query": q])
    answer = rag_result['result']
    context = " ".join([doc.page_content for doc in rag_result['source_documents']])

    # 평가
    eval_result = simple_evalute(q, answer, ref, context)
    print(f"   문제 {i+1} 평가 완료")
    results.append({
        'question': q,
        'answer': answer,
        'evaluation': eval_result
    })

print("\n✅ 평가 완료!")
for r in results:
    print(f"\n📝 Q: {r['question'][:50]}...")
    print(f"   평가: {r['evaluation']}")


#### 🔍 결과 분석 및 개선 방향

|**문제 상황**|**낮은 메트릭**|**개선 방향**|
|----|----|----|
|할루시네이션 발생|Faithfulness ↓|프롬프트에 "Context에 없으면 모른다고 답변" 지시 추가|
|엉뚱한 답변|Answer Relevancy ↓|쿼리 재작성(Query Rewrite) 도입|
|관련 문서 누락|Context Recall ↓|Retriever k값 증가, Hybrid 가중치 조정|
|노이즈 문서 상위|Context Precision ↓|Reranker 모델 교체, Fine-tuning|

---

#### 9. 전체 파이프라인 비교 실헝 (Naive vs Advanced)
지금까지 구축한 다양한 파이프라인의 성능을 한눈에 비교합니다.

In [ ]:
# 다양한 Retriever 파이프라인 정의
pipelines = {
    "1. BM25 Only (Baseline)": bm25_retriever,
    "2. Vector Only (Naive RAG)": vector_retriever,
    "3. Hybrid (BM25 + Vector)": hybrid_retriever,
    "4. Hybrid + Reranker (Advanced)": reranker_retriever
}

test_query = "인공지능 기술의 발전과 활용 사례에 대해 알려줘"

print(f"🔍 테스트 질문: '{test_query}'\n")
print("=" * 80)

comparison_results = []

for name, retriever in pipelines.items():
    docs = retriever.invoke(test_query)

    print(f"\n📌 {name}")
    print(f"   검색된 문서 수: {len(docs)}")

    if docs:
        print(f"   Top-1 미리보기: {docs[0].page_content[:60]}...")
        comparison_results.append({
            "Pipeline": name,
            "문서 수": len(docs),
            "Top-1 출처": docs[0].metadata.get('source', 'unknown')
        })

print("\n" + "=" * 80)
print("\n📊 파이프라인 비교 요약:")
display(pd.DataFrame(comparison_results))

In [ ]:
# 다양한 Retriever 파이프라인 정의 및 LLM as Judge 평가
prpilines = {
    "1. BM25 Only (Baseline)": bm25_retriever,
    "2. Vector Only (Naive RAG)": vector_retriever,
    "3. Hybrid (BM25 + Vector)": hybrid_retriever,
    "4. Hybrid + Reranker (Advanced)": rerank_retriever
}

test_query = "인공지능 기술의 발전과 활용 사례에 대해 알려줘"

print(f"🔍 테스트 질문: '{test_query}'\n")
print("=" * 80)

comparison_results = []
llm_eval_results = []

for name, retriever in pipelines.items():
    docs = retriever.invoke(test_query)
    answer = ""
    context = ""
    if docs:
        # RAG 답변 생성 (예시: 첫 문서 내용 활용)
        context = " ".join([doc.page_content for doc in docs])
        # 실제 RAG 파이프라인이 있다면 여기에 연결
        answer = docs[0].page_content[:300]
    else:
        answer = "검색 결과 없음"
        context = ""
    print(f"\n📌 {name}")
    print(f"   검색된 문서 수: {len(docs)}")
    if docs:
        print(f"   Top-1 미리보기: {docs[0].page_content[:60]}...")
        comparison_results.append({
            "Pipeline": name,
            "문서 수": len(docs),
            "Top-1 출처": docs[0].metadata.get('source', 'unknown')
        })
    # LLM as Judge 평가
    eval_result = simple_evaluate(test_query, answer, "(참조 없응)", context)
    print(f"   LLM 평가: {eval_result}")
    llm_eval_results.append({
        "Pipeline": name,
        "LLM 평가": eval_result
    })

print("\n" + "=" * 80)
print("\n📊 파이프라인 비교 요약:")
display(pd.DataFrame(comparison_results))
print("\n🤖 LLM as Judge 평가 요약:")
display(pd.DataFrame(llm_eval_results))

---

## 🎉 결론 및 고찰
#### 오늘 배운 핵심 개념
1. **검색 방식의 진화**
- BM25 (키워드) → Vector (의미) → Hybrid (결합) → Rerank (정밀화)
2. **Query Decomposition**
- 복잡한 질문을 분해하여 검색 품질 향상
- PPT의 "삼성전자 vs 하이닉스" 예시처럼 실무에서 자주 사용
3. **2-Stage Retrieval**
- Bi-Encoder로 빠르게 후보 추출 → Cross-Encoder로 정밀 재정렬
4. **RAGAS 평가**
- Faithfulness: 할루시네이션 체크
- Answer Relevancy: 질문 의도 부합도
- Context Recall/Precision: 검색 품질

#### PPT의 핵심 메시지
```
"엔지니어링에 무조건적인 해답은 없지만, 목적에 맞는 최적화는 존재합니다."
```